In [1]:
import cv2
import numpy as np
import os 
import matplotlib.pyplot as plt
import time
import mediapipe as mp

In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic
mp_face_mesh = mp.solutions.face_mesh
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands


In [ ]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [ ]:
def draw_styled_landmarks(image, results):
    # Draw Face Landmarks (very light)
    if results.face_landmarks:
        mp_drawing.draw_landmarks(
            image, results.face_landmarks, mp_face_mesh.FACEMESH_TESSELATION,
            mp_drawing.DrawingSpec(color=(100, 100, 255), thickness=1, circle_radius=1),
            mp_drawing.DrawingSpec(color=(100, 100, 255), thickness=1, circle_radius=1))

    # Draw Pose Landmarks (thin and minimal)
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(
            image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=1, circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 200, 0), thickness=1, circle_radius=1))

    # Draw Left Hand Landmarks
    if results.left_hand_landmarks:
        mp_drawing.draw_landmarks(
            image, results.left_hand_landmarks, mp_hands.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(200, 0, 200), thickness=1, circle_radius=1),
            mp_drawing.DrawingSpec(color=(150, 0, 150), thickness=1, circle_radius=1))

    # Draw Right Hand Landmarks
    if results.right_hand_landmarks:
        mp_drawing.draw_landmarks(
            image, results.right_hand_landmarks, mp_hands.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 140, 255), thickness=1, circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 100, 200), thickness=1, circle_radius=1))


In [ ]:
# ## Access our webcam
# cap = cv2.VideoCapture(0)
# with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
#     while cap.isOpened():

#         ret, frame = cap.read()

#         image, results = mediapipe_detection(frame,holistic)
       
#         draw_styled_landmarks(image, results)

#         cv2.imshow('Language Recognizer', image)
#         if cv2.waitKey(10) & 0xFF == ord('q'):
#             break
#     cap.release()
#     cv2.destroyAllWindows()

In [ ]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    return np.concatenate([pose, face, lh, rh])


In [ ]:
DATA_PATH = os.path.join('MP_DATA')
actions = np.array(['hello', 'thanks', 'iloveyou', 'eat'])

# Thirty Videos worth of data
no_sequences = 30
sequence_length = 30

In [ ]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [ ]:
## Access our webcam
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    for action in actions:
        for sequence in range(no_sequences):
            for frame_num in range(sequence_length):

                ret, frame = cap.read()

                image, results = mediapipe_detection(frame,holistic)
            
                draw_styled_landmarks(image, results)

                if frame_num == 0:
                    start_time = time.time()
                    while time.time() - start_time < 2:
                        ret, frame = cap.read()
                        image, results = mediapipe_detection(frame, holistic)
                        cv2.putText(image, 'STARTING COLLECTION', (120,200),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 4, cv2.LINE_AA)
                        cv2.putText(image, f'Collecting frames for {action} Video Number {sequence}', (15,12),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1, cv2.LINE_AA)
                        cv2.imshow('Language Recognizer', image)
                        if cv2.waitKey(1) & 0xFF == ord('q'):
                            break
                else:
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1, cv2.LINE_AA)
                    
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)


                cv2.imshow('Language Recognizer', image)

                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break


    cap.release()
    cv2.destroyAllWindows()

NameError: name 'cv2' is not defined

In [ ]:
cap.release()
cv2.destroyAllWindows()

In [ ]:
result_test = extract_keypoints(results)
np.save('0', result_test)

In [ ]:
np.load('0.npy')

array([ 0.46973997,  0.52889818, -0.53482282, ...,  0.38594657,
        0.56827694, -0.0095707 ])

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical ## used for one hot encoding


In [ ]:
label_map = {label:num for num, label in enumerate(actions)}
label_map

{'hello': 0, 'thanks': 1, 'iloveyou': 2, 'eat': 3}

In [ ]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [ ]:
np.array(sequences).shape ## 120 videos with 30 frame each with 1662 landmarks each

(120, 30, 1662)

In [ ]:
np.array(labels).shape

(120,)

In [ ]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

### Build and Train LSTM Neural Network

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape = (30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

c:\python 3.11\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 69ms/step - categorical_accuracy: 0.2036 - loss: 1.7816
Epoch 2/2000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - categorical_accuracy: 0.3332 - loss: 9.3537
Epoch 3/2000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - categorical_accuracy: 0.2783 - loss: 14.5481
Epoch 4/2000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - categorical_accuracy: 0.2153 - loss: 9.2609
Epoch 5/2000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - categorical_accuracy: 0.2481 - loss: 8.4351
Epoch 6/2000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - categorical_accuracy: 0.2081 - loss: 4.2968
Epoch 7/2000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - categorical_accuracy: 0.2098 - loss: 2.9713
Epoch 8/2000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - categorical_accuracy: 0.2480 - loss: 3.2406
Epoch 9/2000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - categorical_accuracy: 0.4667 - loss: 1.5801
Epoch 10/2000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - categorical_accuracy: 0.5566 - loss: 1.3263
Epoch 11/2000
4/4 ━━━━━━━━━

In [ ]:
res = model.predict(X_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step


In [ ]:
actions[np.argmax(res[0])]

'thanks'

In [ ]:
# model.save('action.h5')
model.load_weights('action.h5')

NameError: name 'model' is not defined

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [ ]:
yhat = model.predict(X_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step


In [ ]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [ ]:
accuracy_score(ytrue, yhat)

0.8333333333333334

In [ ]:
def prob_viz(res, actions, input_frame, colors=None):
    output_frame = input_frame.copy()
    
    # Generate colors if not provided or not enough
    if colors is None or len(colors) < len(actions):
        colors = [(int(255*np.random.rand()), int(255*np.random.rand()), int(255*np.random.rand())) for _ in range(len(actions))]

    for num, prob in enumerate(res[:len(actions)]):  # Safe limit
        cv2.rectangle(output_frame, (0, 60 + num*40), (int(prob * 100), 120 + num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 110 + num*40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    
    return output_frame


In [ ]:
sequence = []
sentence = []
threshold = 0.7
res = np.array([0, 0, 0])  # Default to zero confidence for 3 actions

cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            continue

        image, results = mediapipe_detection(frame, holistic)
        draw_styled_landmarks(image, results)

        keypoints = extract_keypoints(results)
        sequence.insert(0, keypoints)
        sequence = sequence[:30]

        # Predict only if we have enough frames
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])  # Optional: can be removed

            if res[np.argmax(res)] > threshold:
                predicted_action = actions[np.argmax(res)]
                if len(sentence) == 0 or predicted_action != sentence[-1]:
                    sentence.append(predicted_action)

        # Limit sentence length to last 5 predictions
        if len(sentence) > 5:
            sentence = sentence[-5:]

        image = prob_viz(res, actions, image, colors)

        # Display the sentence on the image
        cv2.rectangle(image, (0, 0), (800, 60), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (10, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        cv2.imshow('Language Recognizer', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


NameError: name 'mp_holistic' is not defined